In [1]:
import os
import numpy as np
from matplotlib.lines import Line2D
from scipy.stats import pearsonr, gaussian_kde, linregress, ttest_ind, sem, zscore
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm
from scipy.stats import percentileofscore
from sklearn.utils.validation import check_random_state
from math import factorial
from more_itertools import distinct_permutations
import statsmodels.api as sm

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

import numpy as np
from scipy.stats import linregress
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, ParameterGrid, train_test_split
from tqdm.notebook import tqdm

import warnings
import random
#from torch.utils.data import SubsetRandomSampler
from sklearn.utils import resample

from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score
from collections import Counter

from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score

warnings.filterwarnings("ignore")



In [2]:
from scipy.stats import chi2

def summarize_feature(df, feature, df_out, stat_cols=(10, 50), df_offset=2):

    d = df[df["Feature"] == feature]
    if d.empty:
        return df_out  

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            combined_p = 1 - chi2.cdf(d[col].mean(), df_chi)
            df_out.loc[feature, f"hl_pvalue_combined_{k}"] = combined_p

    # --- OR summary ---
    if "OR" in d.columns:
        or_mean = d["OR"].mean()
        or_std = d["OR"].std(ddof=1)
        df_out.loc[feature, "OR"] = or_mean
        df_out.loc[feature, "2.5%"] = or_mean - or_std
        df_out.loc[feature, "97.5%"] = or_mean + or_std

    # --- min/max de los IC originales (tal como lo estabas haciendo) ---
    if "2.5%" in d.columns:
        df_out.loc[feature, "minOR"] = d["2.5%"].min()
    if "97.5%" in d.columns:
        df_out.loc[feature, "maxOR"] = d["97.5%"].max()

    return df_out


features = ["Mono", "One", "Two", "Three", "Total"] 




def hosmer_lemeshow(y_true, y_prob, g):
    hl_df = pd.DataFrame({
        "observed": y_true,
        "predicted_probability": y_prob
    }).dropna()


    hl_df["group"] = pd.qcut(hl_df["predicted_probability"], g, duplicates="drop")

    hl_table = hl_df.groupby("group").apply(
        lambda x: pd.Series({
            "observed": x["observed"].sum(),
            "expected": x["predicted_probability"].sum(),
            "total": len(x)
        })
    )

    hl_table["observed_neg"] = hl_table["total"] - hl_table["observed"]
    hl_table["expected_neg"] = hl_table["total"] - hl_table["expected"]


    hl_statistic = (
        ((hl_table["observed"] - hl_table["expected"])**2) / hl_table["expected"] +
        ((hl_table["observed_neg"] - hl_table["expected_neg"])**2) / hl_table["expected_neg"]
    ).sum()


    dof = hl_table.shape[0] - 2
    p_value = 1 - chi2.cdf(hl_statistic, dof)

    return hl_statistic, p_value



def summarize_feature_by_covar(df, covar, feature, df_out, stat_cols=(10, 50), df_offset=2):
    d = df[(df["Covar"] == covar) & (df["Feature"] == feature)]
    if d.empty:
        return df_out

    idx = (covar, feature)

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            df_out.loc[idx, f"hl_pvalue_combined_{k}"] = 1 - chi2.cdf(d[col].mean(), df_chi)

    or_mean = d["OR"].mean()
    or_std  = d["OR"].std(ddof=1)
    df_out.loc[idx, "OR"] = or_mean
    df_out.loc[idx, "2.5%"] = or_mean - or_std
    df_out.loc[idx, "97.5%"] = or_mean + or_std

    df_out.loc[idx, "minOR"] = d["2.5%"].min()
    df_out.loc[idx, "maxOR"] = d["97.5%"].max()

    df_out.loc[idx, "n_iter"] = d.shape[0]

    return df_out


## Load data

In [3]:
data = pd.read_csv('data/BBAG-cross.csv')

In [4]:
vars_ = ['Mono', 'One',	'Two',	'Three', 'Total']


## Odd ratios with macrosocials (new vars)

### Without Iteration

In [5]:
results_merge_df_all = data.copy()

covar_list = (
                'GDP_per_capita', 'GDP', 'Educ_doct', 'Educ_bach'
)

df_directions_odd_all = []
df_directions_cov_odd_all = []

for covar in covar_list:
    for i in vars_:

        c_results_merge_df_all = results_merge_df_all[[i, covar, 'GAP_bin']].dropna().reset_index(drop = True)
        
        y_ols = c_results_merge_df_all['GAP_bin']
        X_ols = c_results_merge_df_all[[i, covar]]

        X_train, X_test, y_train, y_test = train_test_split(
            X_ols, y_ols, test_size=0.2, random_state=42
        )

        scaler = MinMaxScaler((0.05, 0.95))

        X_train_scaled = pd.DataFrame(
            scaler.fit_transform(X_train),
            columns=X_train.columns,
            index=X_train.index
        )
        X_test_scaled = pd.DataFrame(
            scaler.transform(X_test),
            columns=X_test.columns,
            index=X_test.index
        )

        X_train_scaled['intercept'] = 1
        X_test_scaled['intercept'] = 1

        model = sm.Logit(y_train, X_train_scaled).fit(disp=0)

        params = model.params
        conf = np.exp(model.conf_int())
        conf['OR'] = np.exp(params)
        conf['z'] = model.tvalues
        conf['P>|z|'] = model.pvalues
        conf.columns = ['2.5%', '97.5%', 'OR', 'z', 'P>|z|']

        df_i = conf.loc[i:i].copy()
        df_i['Feature'] = i
        df_i['Covar'] = covar
        df_directions_odd_all.append(df_i)

        df_c = conf.loc[covar:covar].copy()
        df_c['Feature'] = f"{i}({covar})"
        df_c['Exposure'] = i          # opcional pero útil
        df_c['Covar'] = covar
        df_directions_cov_odd_all.append(df_c)

df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)
df_directions_cov_odd = pd.concat(df_directions_cov_odd_all, axis=0).reset_index(drop=True)

#df_directions_odd, df_directions_cov_odd


In [6]:
for covar in covar_list:
    c_dfOR = df_directions_odd[df_directions_odd.Covar == covar]
    c_dfOR.to_excel('Results/cross/cross_OR_-with-slovakia_new-covar-' + covar + '.xlsx')

    print(covar)
    print('Results/cross/cross_OR_-with-slovakia_new-covar-' + covar + '.xlsx')
    display(c_dfOR)

GDP_per_capita
Results/cross/cross_OR_-with-slovakia_new-covar-GDP_per_capita.xlsx


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
0,1.791987,2.049279,1.916320,19.003325,1.600750e-80,Mono,GDP_per_capita
1,0.591514,0.694690,0.641030,-10.841656,2.184802e-27,One,GDP_per_capita
2,0.510367,0.593270,0.550259,-15.557071,1.424714e-54,Two,GDP_per_capita
3,0.843387,1.002564,0.919538,-1.901901,5.718411e-02,Three,GDP_per_capita
4,0.488009,0.558152,0.521903,-18.980274,2.482993e-80,Total,GDP_per_capita


GDP
Results/cross/cross_OR_-with-slovakia_new-covar-GDP.xlsx


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
5,2.182891,2.476349,2.324995,26.220396,1.555784e-151,Mono,GDP
6,0.742215,0.870378,0.803746,-5.376344,7.601365e-08,One,GDP
7,0.433825,0.504123,0.467655,-19.837762,1.405782e-87,Two,GDP
8,0.552433,0.644908,0.596882,-13.069386,4.926124e-39,Three,GDP
9,0.403717,0.458035,0.430019,-26.206972,2.213088e-151,Total,GDP


Educ_doct
Results/cross/cross_OR_-with-slovakia_new-covar-Educ_doct.xlsx


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
10,1.870533,2.270587,2.060876,14.625381,1.934861e-48,Mono,Educ_doct
11,0.748378,0.916165,0.828032,-3.656683,2.554997e-04,One,Educ_doct
12,0.500896,0.610996,0.553213,-11.679575,1.621044e-31,Two,Educ_doct
13,0.623647,0.803265,0.707781,-5.352845,8.658193e-08,Three,Educ_doct
14,0.440220,0.534352,0.485008,-14.637343,1.622877e-48,Total,Educ_doct


Educ_bach
Results/cross/cross_OR_-with-slovakia_new-covar-Educ_bach.xlsx


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
15,1.879422,2.260557,2.061199,15.354887,3.284798e-53,Mono,Educ_bach
16,0.654446,0.790822,0.719410,-6.820034,9.101866e-12,One,Educ_bach
17,0.537241,0.674306,0.601885,-8.757886,1.989467e-18,Two,Educ_bach
18,0.712226,0.855662,0.780656,-5.290219,1.221698e-07,Three,Educ_bach
19,0.442580,0.532320,0.485381,-15.346861,3.717443e-53,Total,Educ_bach


### 1000-Iteration

In [ ]:

results_merge_df_all = data.copy()

covar_list = (
                'GDP_per_capita', 'GDP', 'Educ_doct', 'Educ_bach'
)


n_boosts = 10  #For performance reasons, the number of iterations was set to 10; however, 1,000 iterations were used for the results reported in the manuscript.”
test_size = 500 / results_merge_df_all.shape[0]


df_directions_odd_all = []
df_directions_cov_all = []


results_dict = {}  

for covar in covar_list:

    for boosts in range(n_boosts):

        for i in vars_:

            c_results_merge_df_all = results_merge_df_all[[i, covar, 'GAP_bin']].dropna().reset_index(drop = True)
            
            y_ols = c_results_merge_df_all['GAP_bin']
            X_ols = c_results_merge_df_all[[i, covar]]

            X_train, X_test, y_train, y_test = train_test_split(
                X_ols, y_ols,
                test_size=test_size,
                stratify=y_ols,
                random_state=boosts
            )

            scaler = MinMaxScaler((0.05, 0.95))

            X_train_scaled = pd.DataFrame(
                scaler.fit_transform(X_train),
                columns=X_train.columns,
                index=X_train.index
            )

            X_test_scaled = pd.DataFrame(
                scaler.transform(X_test),
                columns=X_test.columns,
                index=X_test.index
            )


            X_train_scaled["intercept"] = 1
            X_test_scaled["intercept"] = 1


            model = sm.Logit(y_train, X_train_scaled).fit(disp=0)
            y_test_pred = model.predict(X_test_scaled)

            key_y = (covar, i, "y")
            key_p = (covar, i, "ypred")
            if key_y not in results_dict:
                results_dict[key_y] = pd.Series(dtype=float)
                results_dict[key_p] = pd.Series(dtype=float)

            results_dict[key_y] = pd.concat([results_dict[key_y], y_test], axis=0)
            results_dict[key_p] = pd.concat([results_dict[key_p], y_test_pred], axis=0)

            params = model.params
            conf = np.exp(model.conf_int())
            conf["OR"] = np.exp(params)
            conf["z"] = model.tvalues
            conf["P>|z|"] = model.pvalues
            conf.columns = ["2.5%", "97.5%", "OR", "z", "P>|z|"]

            hl10_stat, hl10_p = hosmer_lemeshow(y_test, y_test_pred, g=10)
            hl50_stat, hl50_p = hosmer_lemeshow(y_test, y_test_pred, g=50)
            hl100_stat, hl100_p = hosmer_lemeshow(y_test, y_test_pred, g=100)

            df_i = conf.loc[i:i].copy()
            df_i["Feature"] = i
            df_i["Covar"] = covar
            df_i["boosts"] = boosts

            df_i["hl_statistic_10"] = hl10_stat
            df_i["hl_pvalue_10"] = hl10_p
            df_i["hl_statistic_50"] = hl50_stat
            df_i["hl_pvalue_50"] = hl50_p
            df_i["hl_statistic_100"] = hl100_stat
            df_i["hl_pvalue_100"] = hl100_p

            df_directions_odd_all.append(df_i)

            df_c = conf.loc[covar:covar].copy()
            df_c["Feature"] = i + "(" + covar + ")"
            df_c["Exposure"] = i
            df_c["Covar"] = covar
            df_c["boosts"] = boosts

            df_c["hl_statistic_10"] = hl10_stat
            df_c["hl_pvalue_10"] = hl10_p
            df_c["hl_statistic_50"] = hl50_stat
            df_c["hl_pvalue_50"] = hl50_p
            df_c["hl_statistic_100"] = hl100_stat
            df_c["hl_pvalue_100"] = hl100_p

            df_directions_cov_all.append(df_c)

df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)
df_directions_cov_odd = pd.concat(df_directions_cov_all, axis=0).reset_index(drop=True)



In [ ]:


features = list(vars_) 
covars = df_directions_odd["Covar"].dropna().unique()

idx = pd.MultiIndex.from_product([covars, features], names=["Covar", "Feature"])
df_directions_odd_f = pd.DataFrame(index=idx)

for c in covars:
    for f in features:
        df_directions_odd_f = summarize_feature_by_covar(
            df_directions_odd, c, f, df_directions_odd_f,
            stat_cols=(10, 50, 100), 
            df_offset=2
        )

df_directions_odd_f = df_directions_odd_f.reset_index()
df_directions_odd_f;


In [ ]:
for covar in covar_list:
    c_dfOR = df_directions_odd_f[df_directions_odd_f.Covar == covar]
    c_dfOR.to_excel('Results/cross/cross_OR_-with-slovakia_new-covar-' + covar + '-iter.xlsx')

    print(covar)
    display(c_dfOR)